# 🎤 TTS Server - ABSOLUTELY NO LIMITS

**Microsoft Edge TTS + Cloudflare Tunnel**

## ✅ Features:
- **NO TEXT LENGTH LIMIT** - any text length works
- **NO TIMEOUTS** - server waits as long as needed
- **16 English voices** (US, UK, Australian, Indian, Canadian)
- **Speed control** (0.5x to 3.0x)
- **CORS enabled** - works with any website
- **GET and POST** endpoints
- **Automatic public URL** via Cloudflare Tunnel

## 🚀 One-click deployment

In [ ]:
# ============================================================================
# 📦 INSTALL DEPENDENCIES
# ============================================================================
print("=" * 80)
print("📦 INSTALLING DEPENDENCIES")
print("=" * 80)

print("Installing edge-tts, Flask and requests...")
!pip install edge-tts flask requests -q

print("✅ Dependencies installed")

In [ ]:
# ============================================================================
# 🚀 LAUNCH TTS SERVER - NO LIMITS AT ALL
# ============================================================================
print("\n" + "=" * 80)
print("🚀 LAUNCHING TTS SERVER - NO LIMITS")
print("=" * 80)

import os
import subprocess
import asyncio
import threading
import time
import re
import sys
import json
from flask import Flask, request, Response, jsonify
import edge_tts

print("✅ Modules imported")

# Create Flask app
app = Flask(__name__)

# Define voices globally
voices = {
    # US Voices
    "us_young": "en-US-ChristopherNeural",
    "us_male": "en-US-EricNeural", 
    "us_male2": "en-US-GuyNeural",
    "us_female": "en-US-JennyNeural",
    "us_female2": "en-US-AriaNeural",
    "us_female3": "en-US-AnaNeural",
    
    # UK Voices
    "uk_male": "en-GB-RyanNeural",
    "uk_male2": "en-GB-AlfieNeural",
    "uk_female": "en-GB-SoniaNeural",
    "uk_female2": "en-GB-LibbyNeural",
    
    # Australian Voices
    "au_male": "en-AU-WilliamNeural",
    "au_female": "en-AU-NatashaNeural",
    
    # Indian Voices
    "indian_male": "en-IN-PrabhatNeural",
    "indian_female": "en-IN-NeerjaNeural",
    
    # Canadian Voices
    "ca_male": "en-CA-LiamNeural",
    "ca_female": "en-CA-ClaraNeural"
}

# Enable CORS
@app.after_request
def cors(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Access-Control-Allow-Methods"] = "GET, POST, OPTIONS"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type, X-Requested-With"
    return response

@app.route("/", methods=["GET", "POST", "OPTIONS"])
def tts_root():
    if request.method == 'OPTIONS':
        return '', 200
    return tts_request()

@app.route("/tts", methods=["GET", "POST", "OPTIONS"])
def tts_request():
    """Main TTS function - ABSOLUTELY NO LIMITS"""
    if request.method == 'OPTIONS':
        return '', 200
    
    # Get parameters
    text = ""
    voice = "us_young"
    speed = "1.0"
    
    if request.method == 'GET':
        text = request.args.get("text", "")
        voice = request.args.get("voice", "us_young")
        speed = request.args.get("speed", "1.0")
    elif request.method == 'POST':
        try:
            if request.is_json:
                data = request.get_json()
                text = data.get('text', '')
                voice = data.get('voice', 'us_young')
                speed = data.get('speed', '1.0')
        except:
            text = ""
    
    if not text:
        return jsonify({"error": "No text provided"}), 400
    
    print(f"📝 Processing: {len(text)} chars, voice={voice}, speed={speed}")
    
    # Get voice ID
    if voice not in voices:
        voice = "us_young"
    voice_id = voices[voice]
    
    # Format speed for edge-tts
    try:
        speed_float = float(speed)
        if speed_float == 1.0:
            rate_str = "+0%"
        else:
            percentage = int((speed_float - 1.0) * 100)
            rate_str = f"+{percentage}%" if percentage >= 0 else f"{percentage}%"
    except:
        rate_str = "+0%"
    
    # Simple async function - NO TIMEOUTS, NO LIMITS
    async def generate_audio():
        communicate = edge_tts.Communicate(text, voice_id, rate=rate_str)
        chunks = []
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                chunks.append(chunk["data"])
        return b"".join(chunks)
    
    try:
        # Use asyncio.run() directly - no wait_for, no timeouts
        audio = asyncio.run(generate_audio())
        
        response = Response(audio, mimetype="audio/mpeg")
        response.headers["Content-Length"] = len(audio)
        response.headers["X-TTS-Length"] = len(text)
        response.headers["X-TTS-Voice"] = voice
        response.headers["X-TTS-Speed"] = speed
        
        print(f"✅ Success: {len(audio)} bytes generated")
        return response
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return jsonify({"error": str(e)}), 500

@app.route("/status", methods=["GET", "OPTIONS"])
def status():
    """Server status endpoint"""
    if request.method == 'OPTIONS':
        return '', 200
    return jsonify({
        "status": "running",
        "message": "TTS Server - NO LIMITS",
        "voices": list(voices.keys()),
        "speed_range": "0.5 to 3.0"
    })

# Server runner function
def run_server():
    app.run(host="0.0.0.0", port=9999, debug=False, threaded=True, use_reloader=False)

# Kill old processes
print("Stopping old processes...")
!fuser -k 9999/tcp 2>/dev/null || true

# Start server
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

print("✅ TTS server running on http://localhost:9999")
print("📋 Available endpoints:")
print("   • GET  /tts?text=...&voice=...&speed=...")
print("   • POST /tts (with JSON)")
print("   • GET  /status")
print("✅ NO LIMITS - no text length limit, no timeout")

In [ ]:
# ============================================================================
# 🌐 SETUP CLOUDFLARED
# ============================================================================
print("\n" + "=" * 80)
print("🌐 SETTING UP CLOUDFLARED")
print("=" * 80)

# Stop old cloudflared processes
!pkill -f cloudflared 2>/dev/null || true
time.sleep(2)

print("Installing cloudflared...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb 2>/dev/null || apt-get install -f -y

# Run cloudflared
print("Starting Cloudflare Tunnel...")
!cloudflared tunnel --url http://localhost:9999 > /tmp/cloudflared.log 2>&1 &
time.sleep(5)

# Get URL
print("Getting URL...")
time.sleep(15)

try:
    with open('/tmp/cloudflared.log', 'r') as f:
        logs = f.read()
    
    public_url = None
    url_patterns = [
        r'https://[a-zA-Z0-9.-]+\.trycloudflare\.com',
        r'Visit it at.*?(https://[^\s]+)',
        r'Your quick Tunnel.*?(https://[^\s]+)'
    ]
    
    for pattern in url_patterns:
        matches = re.findall(pattern, logs)
        if matches:
            public_url = matches[-1]
            print(f"✅ URL found: {public_url}")
            break
    
    if not public_url:
        public_url = "https://your-server.trycloudflare.com"
        print("⚠️ URL not found in logs")
        
except Exception as e:
    print(f"Error reading logs: {e}")
    public_url = None

print("✅ Cloudflare Tunnel ready")

In [ ]:
# ============================================================================
# 🎯 READY
# ============================================================================
if public_url:
    print("\n" + "=" * 80)
    print("🎯 SERVER READY - NO LIMITS")
    print("=" * 80)
    print(f"\n🔗 YOUR URL: {public_url}")
    
    # Save URL
    with open('/content/tts_url.txt', 'w') as f:
        f.write(public_url)
    print(f"💾 URL saved to /content/tts_url.txt")
    
    print("\n📝 SIMPLE USAGE:")
    print(f"GET:  {public_url}/tts?text=Your+text&voice=us_male&speed=1.5")
    
    print("\n✅ NO TEXT LENGTH LIMIT")
    print("✅ NO TIMEOUTS")
    print("✅ Works with any text length")
    
else:
    print("\n❌ Failed to get URL")
    print("Check the logs above for https://")

print("\n" + "=" * 80)
print("✅ SERVER IS RUNNING")
print("=" * 80)
print("\n💡 Keep this Colab running to use the server")